In [3]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
# CPU will tak 50s infer
# !pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user


In [10]:
# Use GPU 3s infer
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable


In [11]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [12]:
#load pdf files
loader = PyPDFDirectoryLoader("./pdf")
data = loader.load()

In [13]:
print(data)

[Document(page_content='Rust Atomics and Locks\n💛Chapter 1. Basics of Rust Concurrency\nLong before multi-core processors were commonplace, operating systems allowed for a\nsingle computer to run many programs concurrently . This is achieved by rapidly switching\nbetween processes, allowing each to repeatedly make a little bit of progress, one by one.\nNowadays, virtuall y all our computer s and even our phones and watches have processors\nwith multiple cores, which can truly execute multiple processes in parallel .\nOperating systems isolate processes from each other as much as possible, allowing a pro‐\ngram to do its thing while completely unaware of what any other processes are doing. For\nexample, a process cannot normally access the memory of another process, or communi ‐\ncate with it in any way , without asking the operating system’ s kernel ﬁrst.\nHowever , a program can spawn extra threads of executi on, as part of the same process .\nThreads within the same process are not i

In [14]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [15]:
len(text_chunks)

28

In [16]:
#get the third chunk
text_chunks[2]

Document(page_content='println!("Hello from the main thread.");\n    t1.join().unwrap();\n    t2.join().unwrap();\n}\nThe .join()  method waits until the thread has ﬁnished executing and returns a std::\nthread:: Result . If the thread did not successfully ﬁnish its function because it panicked,\nthis will contain the panic message. We could attempt to handle that situation, or just call\n.unwrap()  to panic when joining a panicked thread.\nRunning this version of our program will no longer result in truncated output:\nHello from the main thread.\nHello from another thread!\nThis is my thread id: ThreadId(3)\nHello from another thread!\nThis is my thread id: ThreadId(2)\nThe only thing that still changes between runs is the order in which the messages are\nprinted :\nHello from the main thread.\nHello from another thread!\nHello from another thread!\nThis is my thread id: ThreadId(2)\nThis is my thread id: ThreadId(3)\nOutput Locking\nThe println  macro uses std::io::Stdout:: lock()  t

In [17]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [18]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [19]:
# connect to google drive if need
# from google.colab import drive
# drive.mount('/content/drive')


In [20]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="./mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096,
    n_gpu_layers=30,
    n_threads=2,
    n_batch=521,
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     

In [21]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [25]:
qa.run("What prevent thread not being execute")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   740.36 ms
llama_print_timings:      sample time =     5.55 ms /    19 runs   (    0.29 ms per token,  3424.04 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  1018.70 ms /    19 runs   (   53.62 ms per token,    18.65 tokens per second)
llama_print_timings:       total time =  1046.58 ms


'\n\nThe use of a join method prevents threads from being executed until they have finished.'

In [23]:
# Wait for input if need
# import sys

# while True:
#   user_input = input(f"Input Prompt: ")
#   if user_input == 'exit':
#     print('Exiting')
#     sys.exit()
#   if user_input == '':
#     continue
#   result = qa({'query': user_input})
#   print(f"Answer: {result['result']}")